## Import packages

In [1]:
# json parsing
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import warnings
warnings.filterwarnings('ignore')

## Read the raw messages from the topic eduAssessment

In [2]:
raw_messages = spark.read.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:29092")\
.option("subscribe","eduAssessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

## Make sure everything is running well: Cache data

In [3]:
raw_messages.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

Ran perfectly, we have our dataframe with the data cached. Good sanity check before starting

## Visualiza the Schema

In [4]:
raw_messages.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



## See the top 5 messages as a DataFrame (binary so far) - Visualization of Data

In [5]:
raw_messages.show(5)

+----+--------------------+--------------+---------+------+--------------------+-------------+
| key|               value|         topic|partition|offset|           timestamp|timestampType|
+----+--------------------+--------------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|eduAssessments|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|eduAssessments|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|eduAssessments|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|eduAssessments|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|eduAssessments|        0|     4|1969-12-31 23:59:...|            0|
+----+--------------------+--------------+---------+------+--------------------+-------------+
only showing top 5 rows



For our analysis, we only care about the key-values pairs, which are encoded as binary by Kafka. All come from the same topic in this example

## Take all the values, and cast them as Strings. Extract the JSON from string messages

In [6]:
messages = raw_messages.select(raw_messages.value.cast('string'))
naive_extracted_messages = messages.rdd.map(lambda x: json.loads(x.value)).toDF()
naive_extracted_messages.show(5)

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

At this moment, the data estracted from JSON is a mess. The main reason being that we receive quite a nested JSON file, not a flat one. Some parts of the dataframe will be used as if, others we will use SparSQL to try to flatten the nests, specially in the Sequences field: where all the questions are stored. Let's first check the Schema of this DF. Also, because from sequences we need integer values (correct, incorrect, etc), we cannot cast to string yet, because those fields will become null

In [7]:
naive_extracted_messages.printSchema()

root
 |-- base_exam_id: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- keen_created_at: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- keen_timestamp: string (nullable = true)
 |-- max_attempts: string (nullable = true)
 |-- sequences: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: boolean (valueContainsNull = true)
 |-- started_at: string (nullable = true)
 |-- user_exam_id: string (nullable = true)



The nest become quite evident on the Sequences Field. Everything else is flat, and can be used as if. One way of solving it, because we're losing important information in the sequences field, is to enfornce a Schema from the data. The implied Schema works on the first level, but it doesn't work for nested levels. I'll enforce a Schema of the values that I care for this analysis: correct and total questions per attempt. The information in the questions map it's important, so we'll keep as is in the implied map, but we'll create a new DF with only the information need it for this analysis

In [8]:
final_schema = StructType([StructField('base_exam_id', StringType(), True),
                     StructField('certification', StringType(), True),
                     StructField('exam_name', StringType(), True),
                     StructField('keen_created_at', StringType(), True),
                     StructField('keen_id', StringType(), True),
                     StructField('keen_timestamp', StringType(), True),
                     StructField('max_attempts', StringType(), True),
                     StructField('sequences', StructType([
                         StructField('attempt', IntegerType(), True),
                         StructField('counts', StructType([
                             StructField('all_correct', StringType(), True),
                             StructField('correct', IntegerType(), True),
                             StructField('total', IntegerType(), True)
                         ]))]))])

In [9]:
important_extracted_messages = messages.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)
important_extracted_messages.show(5)

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+---------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|      sequences|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+---------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|[1,[false,2,4]]|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|[1,[false,1,4]]|
|4beeac16-bb83-4d5...|        false|The Principles of...| 1516738973.653394|5a67999d3ed3e3000...| 1516738973.653394|         1.0|[1,[false,3,4]]|
|4beeac16-bb83-4d5...|        false|The Principles of...|1516738921.1137421|5a6799694fc7c7000...|1516738921.1137421|        

Here we only have the important fields that we need, after enforcing a Schema on the JSON file. Now we can do SQL with the grades

## Extraction using SparkSQL

In [10]:
# Create a Temp Table
important_extracted_messages.registerTempTable('messages')
naive_extracted_messages.registerTempTable('questions')

In [11]:
spark.sql("select exam_name, count(exam_name) as total from messages group by exam_name order by total desc").show()

+--------------------+-----+
|           exam_name|total|
+--------------------+-----+
|        Learning Git|  394|
|Introduction to P...|  162|
|Introduction to J...|  158|
|Intermediate Pyth...|  158|
|Learning to Progr...|  128|
|Introduction to M...|  119|
|Software Architec...|  109|
|Beginning C# Prog...|   95|
|    Learning Eclipse|   85|
|Learning Apache M...|   80|
|Beginning Program...|   79|
|       Mastering Git|   77|
|Introduction to B...|   75|
|Advanced Machine ...|   67|
|Learning Linux Sy...|   59|
|JavaScript: The G...|   58|
|        Learning SQL|   57|
|Practical Java Pr...|   53|
|    HTML5 The Basics|   52|
|   Python Epiphanies|   51|
+--------------------+-----+
only showing top 20 rows



Using the JSON encoding and the tree structure, we can use SQL to extrat any values that we need inside the DataFrame. This last SQL command is the same extraction that I did using Pandas and dictionaries on the last Assignment, but in one line of code 

## Extracting the Grades for each assignment

In [12]:
spark.sql("select exam_name, round(avg((sequences.counts.correct)*100/(sequences.counts.total)),2) as avg_grades, \
            round(stddev((sequences.counts.correct)*100/(sequences.counts.total)),2) as std_grades \
           from messages group by exam_name order by avg_grades").show()

+--------------------+----------+----------+
|           exam_name|avg_grades|std_grades|
+--------------------+----------+----------+
|Example Exam For ...|      null|      null|
|Client-Side Data ...|      20.0|     28.28|
|       View Updating|      25.0|      50.0|
|Native Web Apps f...|      25.0|       NaN|
|Arduino Prototypi...|     33.33|     23.57|
|Mastering Advance...|     36.03|     32.68|
|           Nullology|      37.5|     51.75|
| Mastering Web Views|     41.67|     52.04|
|Building Web Serv...|     41.67|     28.87|
|Web & Native Work...|     41.67|     23.57|
|Cloud Computing W...|     42.65|     38.29|
|         Offline Web|     43.59|     34.39|
|Learning C# Best ...|     46.29|     26.02|
|Design Patterns i...|     46.67|     34.36|
|Software Architec...|     47.94|     28.29|
|  Learning Java EE 7|      48.0|     32.03|
|Data Visualizatio...|     49.19|     34.45|
|Being a Better In...|      50.0|     47.14|
|Learning Data Mod...|      50.0|     27.95|
|Learning 

This is the average grades for each course, in %. This type of statistic is very important for us to determine how our students are doing on their assignments, and later can be done dynamically. We also add the standard deviation. 

## Extract all the tables to HDFS

### Raw Data

In [14]:
raw_messages.write.parquet('/tmp/raw_data1')

### Naive Extracted JSON - for Data regarding the pool of Questions

In [15]:
a = naive_extracted_messages.select(explode(naive_extracted_messages.sequences))
b = a.select(explode(a.value))
questions = b.select(explode(b.col))
questions.show(5)

+---------------+-----+
|            key|value|
+---------------+-----+
|user_incomplete| true|
|    user_result| null|
| user_submitted| true|
|        options| null|
|   user_correct|false|
+---------------+-----+
only showing top 5 rows



In [16]:
questions.write.parquet('/tmp/questions_pool_data1')

To extract the question pool, we needed to break the maps created by the implied Schema. I did that using explode two times, and then store it as question pool

### Grades for each assignment by ID and by Course Name

In [17]:
#ID
grades_ID = spark.sql("select base_exam_id, (sequences.counts.correct)*100/(sequences.counts.total) as grades \
           from messages")
grades_ID.write.parquet('/tmp/grades_ID1')
#Course
course_ID = spark.sql("select exam_name, round(avg((sequences.counts.correct)*100/(sequences.counts.total)),2) as avg_grades, \
            round(stddev((sequences.counts.correct)*100/(sequences.counts.total)),2) as std_grades \
           from messages group by exam_name order by avg_grades")
course_ID.write.parquet('/tmp/course_ID1')